In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/.shortcut-targets-by-id/1qNzMmGHCg5Xa63Vw3aKbZdMXvkfT2CgC/MedStar/MS_CODE/FieldNN


# Read Raw Data

In [1]:
import os 
import pandas as pd

RecName = 'Smoking'
folder = 'data/Simulations/'
raw_file_path =  os.path.join(folder, RecName) + '.csv'
print(raw_file_path)


data/Simulations/Smoking.csv


In [2]:

# load all records
df = pd.read_csv(raw_file_path)
df

,PID,ECID,RID,DT,V
0,P0,EC23,15471,2023-03-19,B
1,P0,EC142,21622,2023-03-20,D
2,P0,EC113,19101,2023-03-23,A
3,P0,EC166,22867,2023-03-26,B
4,P0,EC91,19104,2023-03-27,D
...,...,...,...,...,...
96,P5,EC129,20285,2023-04-16,B
97,P6,EC92,14290,2023-03-19,D
98,P6,EC123,17942,2023-03-21,A
99,P6,EC15,17943,2023-03-28,B


# Preprocess

## Add Missing Records

In [3]:
from recfldgrn.datapoint import load_df_data_from_folder

prefix_cols = ['PID', 'ECID']
focal_cols = ['RID']

Parent_RecName = 'EC'
rec_folder = 'data/ProcData/RecFolder/'
fullrec_folder =  os.path.join(rec_folder, Parent_RecName) 
df_parent = load_df_data_from_folder(fullrec_folder)
df_parent

,PID,ECID,DT_min,DT_max,BasicInfo
0,P0,EC18,2023-03-18,2023-04-16,D&X
1,P0,EC140,2023-03-19,2023-04-05,D&X
2,P0,EC23,2023-03-19,2023-04-16,C&W
3,P0,EC141,2023-03-20,2023-04-16,C&Z
4,P0,EC142,2023-03-20,2023-04-15,B&W
...,...,...,...,...,...
220,P6,EC15,2023-03-21,2023-04-09,B&X
221,P6,EC93,2023-03-23,2023-04-04,B&X
222,P6,EC195,2023-03-25,2023-03-25,C&Z
223,P6,EC173,2023-04-05,2023-04-13,C&W


In [4]:
import numpy as np

d = pd.merge(df_parent[prefix_cols], df, on = prefix_cols, how = 'left')
for col in focal_cols: 
    d.loc[d[col].isnull(), col] = np.ones(d[col].isnull().sum()).cumsum() + d[col].max()
    d[col] = d[col].astype(int)
    assert len(d[col]) == d[col].nunique()
    
df = d
df

,PID,ECID,RID,DT,V
0,P0,EC18,22869,NaN,NaN
1,P0,EC140,20414,2023-04-01,B
2,P0,EC23,15471,2023-03-19,B
3,P0,EC23,17928,2023-04-15,B
4,P0,EC141,22870,NaN,NaN
...,...,...,...,...,...
226,P6,EC15,17943,2023-03-28,B
227,P6,EC93,15484,2023-04-04,A
228,P6,EC195,22996,NaN,NaN
229,P6,EC173,22997,NaN,NaN


## Update DT Format

In [5]:
df['DT'] = pd.to_datetime(df['DT'])

# Save To Disk by Bucket

In [6]:
from recfldgrn.datapoint import convert_PID_to_PIDgroup
from recfldgrn.datapoint import RANGE_SIZE, write_df_to_folders, load_df_data_from_folder

In [7]:
rec_folder = 'data/ProcData/RecFolder'
fullrec_folder = os.path.join(rec_folder, RecName)
print(df.shape, fullrec_folder)

if not os.path.exists(fullrec_folder): os.makedirs(fullrec_folder)
write_df_to_folders(RecName, fullrec_folder, df)


(231, 5) data/ProcData/RecFolder/Smoking


In [8]:
from recfldgrn.datapoint import PatientDP
print(rec_folder)
PID = 'P4'
Pat = PatientDP(PID, rec_folder, RANGE_SIZE)
print(Pat)
dfx = Pat.get_df_rec(RecName)
dfx

data/ProcData/RecFolder


,PID,ECID,RID,DT,V
0,P4,EC125,22970,NaT,NaN
1,P4,EC24,16452,2023-03-31,A
2,P4,EC24,21387,2023-04-13,C
3,P4,EC36,22971,NaT,NaN
4,P4,EC80,22972,NaT,NaN
5,P4,EC163,22973,NaT,NaN
6,P4,EC106,22974,NaT,NaN
7,P4,EC85,22975,NaT,NaN
8,P4,EC88,22589,2023-04-12,D
9,P4,EC179,14005,2023-03-22,B


In [9]:
from recfldgrn.datapoint import load_df_data_from_folder
print(fullrec_folder)
df = load_df_data_from_folder(fullrec_folder)
df

data/ProcData/RecFolder/Smoking


,PID,ECID,RID,DT,V
0,P0,EC18,22869,NaT,NaN
1,P0,EC140,20414,2023-04-01,B
2,P0,EC23,15471,2023-03-19,B
3,P0,EC23,17928,2023-04-15,B
4,P0,EC141,22870,NaT,NaN
...,...,...,...,...,...
226,P6,EC15,17943,2023-03-28,B
227,P6,EC93,15484,2023-04-04,A
228,P6,EC195,22996,NaT,NaN
229,P6,EC173,22997,NaT,NaN


# Field & Grain

In [10]:
FieldGrainInfo = []

In [14]:
RecName = 'Smoking'
prefix_ids = ['PID', 'ECID']
focal_ids = ['RID']
field_list = ['V', 'DT']

## Field `V`

In [15]:
# (a) field and (b) get field-focused df
field = 'V'
dfx = df[prefix_ids + focal_ids + [field]]
dfx

,PID,ECID,RID,V
0,P0,EC18,22869,NaN
1,P0,EC140,20414,B
2,P0,EC23,15471,B
3,P0,EC23,17928,B
4,P0,EC141,22870,NaN
...,...,...,...,...
226,P6,EC15,17943,B
227,P6,EC93,15484,A
228,P6,EC195,22996,NaN
229,P6,EC173,22997,NaN


### Grain `'Smoking@V-SmokingDftGrn'`

In [16]:
from recfldgrn.grains import Field2Grain_2_get_GrnStrList_Fn
from recfldgrn.grains import Field2Grain_2_get_GrnIdxList_Fn
from recfldgrn.grains import Field2Grain_2_get_GrnWgtList_Fn
from recfldgrn.utils import generate_grain_vocab_info

In [20]:
# (c) grain
grain = 'SmokingDftGrn'
field2grain = '-'.join([field, grain])
recfield2grain = RecName + '@' + field2grain
print(grain)
print(field2grain)
print(recfield2grain)

SmokingDftGrn
V-SmokingDftGrn
Smoking@V-SmokingDftGrn


In [16]:
# grain_str_fn = Field2Grain_2_get_GrnStrList_Fn[recfield2grain]
# s = dfx[field].apply(lambda x: grain_str_fn(x))
# s

In [17]:
# Vocab = generate_grain_vocab_info(s) # load from disk
# v2idx = Vocab['v2idx']
# v2idx

In [18]:
# grain_idx_fn = Field2Grain_2_get_GrnIdxList_Fn[recfield2grain]
# s = dfx[field].apply(lambda x: grain_idx_fn(x, v2idx))
# s

In [21]:
from recfldgrn.grains.misc.dt_grn import load_DTgrn_Dictionary
from transformers import AutoTokenizer

if 'Nume' in grain:
    print('Current grain is a numeric grain')
    grain_wgt_fn = Field2Grain_2_get_GrnWgtList_Fn[recfield2grain]
    series_wgt = dfx[field].apply(grain_wgt_fn)
    Vocab = {'v2idx': len(series_wgt.iloc[0]) * [1]}
    v2idx = Vocab['v2idx']

elif 'DT' in grain:
    print('Current grain is a datetime grain')
    grain_str_fn = Field2Grain_2_get_GrnStrList_Fn[recfield2grain]
    series_str = dfx[field].apply(grain_str_fn)
    Vocab = load_DTgrn_Dictionary()
    v2idx = Vocab['v2idx']
    
elif 'Tknz' in grain:
    print('Current grain is a tokenize grain')
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    v2idx = tokenizer
    Vocab = {'v2idx': v2idx}

else:
    grain_str_fn = Field2Grain_2_get_GrnStrList_Fn[recfield2grain]
    series_str = dfx[field].apply(grain_str_fn)
    Vocab = generate_grain_vocab_info(series_str)
    v2idx = Vocab['v2idx']

print(v2idx)
grain_idx_fn = Field2Grain_2_get_GrnIdxList_Fn[recfield2grain]
series_idx = dfx[field].apply(lambda x: grain_idx_fn(x, v2idx))
series_idx


{'_padding': 0, '_missing': 1, 'A': 2, 'B': 3, 'D': 4, 'C': 5}


0      [1]
1      [3]
2      [3]
3      [3]
4      [1]
      ... 
226    [3]
227    [2]
228    [1]
229    [1]
230    [1]
Name: V, Length: 231, dtype: object

In [23]:
series_idx

0      [1]
1      [3]
2      [3]
3      [3]
4      [1]
      ... 
226    [3]
227    [2]
228    [1]
229    [1]
230    [1]
Name: V, Length: 231, dtype: object

In [24]:
d = {}
d['RecName'] = RecName

d['prefix_ids'] = prefix_ids
d['focal_ids'] = focal_ids 
d['field_list'] = field_list

d['field'] = field
d['grain'] = grain
d['field2grain'] = field2grain
d['recfield2grain'] = recfield2grain
d['Vocab'] = Vocab

FieldGrainInfo.append(d)

In [25]:
d

{'RecName': 'Smoking',
 'prefix_ids': ['PID', 'ECID'],
 'focal_ids': ['RID'],
 'field_list': ['V', 'DT'],
 'field': 'V',
 'grain': 'SmokingDftGrn',
 'field2grain': 'V-SmokingDftGrn',
 'recfield2grain': 'Smoking@V-SmokingDftGrn',
 'Vocab': {'idx2v': {0: '_padding',
   1: '_missing',
   2: 'A',
   3: 'B',
   4: 'D',
   5: 'C'},
  'v2idx': {'_padding': 0, '_missing': 1, 'A': 2, 'B': 3, 'D': 4, 'C': 5},
  'v2freq': {'_padding': 0,
   '_missing': 130,
   'A': 29,
   'B': 27,
   'D': 24,
   'C': 21}}}

## Field `DT_min`

In [26]:
# (a) field and (b) get field-focused df
field = 'DT'
dfx = df[prefix_ids + focal_ids + [field]]
dfx

,PID,ECID,RID,DT
0,P0,EC18,22869,NaT
1,P0,EC140,20414,2023-04-01
2,P0,EC23,15471,2023-03-19
3,P0,EC23,17928,2023-04-15
4,P0,EC141,22870,NaT
...,...,...,...,...
226,P6,EC15,17943,2023-03-28
227,P6,EC93,15484,2023-04-04
228,P6,EC195,22996,NaT
229,P6,EC173,22997,NaT


### DTGrn

In [27]:
# (c) grain
grain = 'DTDftGrn'
field2grain = '-'.join([field, grain])
recfield2grain = RecName + '@' + field2grain
print(grain)
print(field2grain)
print(recfield2grain)

DTDftGrn
DT-DTDftGrn
Smoking@DT-DTDftGrn


In [29]:
from recfldgrn.grains.misc.dt_grn import load_DTgrn_Dictionary
from transformers import AutoTokenizer

if 'Nume' in grain:
    # print('Current grain is a numeric grain')
    grain_wgt_fn = Field2Grain_2_get_GrnWgtList_Fn[recfield2grain]
    series_wgt = dfx[field].apply(grain_wgt_fn)
    Vocab = {'v2idx': len(series_wgt.iloc[0]) * [1]}
    v2idx = Vocab['v2idx']

elif 'DT' in grain:
    grain_str_fn = Field2Grain_2_get_GrnStrList_Fn[recfield2grain]
    series_str = dfx[field].apply(grain_str_fn)
    Vocab = load_DTgrn_Dictionary()
    v2idx = Vocab['v2idx']
    
elif 'Tknz' in grain:
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    v2idx = tokenizer
    Vocab = {'v2idx': v2idx}

else:
    grain_str_fn = Field2Grain_2_get_GrnStrList_Fn[recfield2grain]
    series_str = dfx[field].apply(grain_str_fn)
    Vocab = generate_grain_vocab_info(series_str)
    v2idx = Vocab['v2idx']

print(v2idx)
grain_idx_fn = Field2Grain_2_get_GrnIdxList_Fn[recfield2grain]
series_idx = dfx[field].apply(lambda x: grain_idx_fn(x, v2idx))
series_idx


{'_padding': 0, '_missing': 1, 'Yx': 2, 'Y2010': 3, 'Y2011': 4, 'Y2012': 5, 'Y2013': 6, 'Y2014': 7, 'Y2015': 8, 'Y2016': 9, 'Y2017': 10, 'Y2018': 11, 'Y2019': 12, 'Y2020': 13, 'Y2021': 14, 'Y2022': 15, 'Y2023': 16, 'Y2024': 17, 'Y2025': 18, 'Y2026': 19, 'Y2027': 20, 'Y2028': 21, 'Y2029': 22, 'Mx': 23, 'M1': 24, 'M2': 25, 'M3': 26, 'M4': 27, 'M5': 28, 'M6': 29, 'M7': 30, 'M8': 31, 'M9': 32, 'M10': 33, 'M11': 34, 'M12': 35, 'Dx': 36, 'D1': 37, 'D2': 38, 'D3': 39, 'D4': 40, 'D5': 41, 'D6': 42, 'D7': 43, 'D8': 44, 'D9': 45, 'D10': 46, 'D11': 47, 'D12': 48, 'D13': 49, 'D14': 50, 'D15': 51, 'D16': 52, 'D17': 53, 'D18': 54, 'D19': 55, 'D20': 56, 'D21': 57, 'D22': 58, 'D23': 59, 'D24': 60, 'D25': 61, 'D26': 62, 'D27': 63, 'D28': 64, 'D29': 65, 'D30': 66, 'D31': 67, 'Sx': 68, 'S1': 69, 'S2': 70, 'S3': 71, 'S4': 72, 'Wx': 73, 'W1': 74, 'W2': 75, 'W3': 76, 'W4': 77, 'W5': 78, 'W6': 79, 'W7': 80, 'WYx': 81, 'WY1': 82, 'WY2': 83, 'WY3': 84, 'WY4': 85, 'WY5': 86, 'WY6': 87, 'WY7': 88, 'WY8': 89, 'WY

0       [2, 23, 36, 68, 73, 81, 137]
1      [16, 27, 37, 70, 79, 95, 138]
2      [16, 26, 55, 69, 80, 93, 138]
3      [16, 27, 51, 70, 79, 97, 138]
4       [2, 23, 36, 68, 73, 81, 137]
                   ...              
226    [16, 26, 64, 69, 75, 95, 138]
227    [16, 27, 40, 70, 75, 96, 138]
228     [2, 23, 36, 68, 73, 81, 137]
229     [2, 23, 36, 68, 73, 81, 137]
230     [2, 23, 36, 68, 73, 81, 137]
Name: DT, Length: 231, dtype: object

In [30]:
d = {}
d['RecName'] = RecName

d['prefix_ids'] = prefix_ids
d['focal_ids'] = focal_ids 
d['field_list'] = field_list


d['field'] = field
d['grain'] = grain
d['field2grain'] = field2grain
d['recfield2grain'] = recfield2grain
d['Vocab'] = Vocab

FieldGrainInfo.append(d)

# Save Field Grain Info



In [31]:
df_FieldGrainInfo = pd.DataFrame(FieldGrainInfo)
df_FieldGrainInfo

,RecName,prefix_ids,focal_ids,field_list,field,grain,field2grain,recfield2grain,Vocab
0,Smoking,"[PID, ECID]",[RID],"[V, DT]",V,SmokingDftGrn,V-SmokingDftGrn,Smoking@V-SmokingDftGrn,"{'idx2v': {0: '_padding', 1: '_missing', 2: 'A..."
1,Smoking,"[PID, ECID]",[RID],"[V, DT]",DT,DTDftGrn,DT-DTDftGrn,Smoking@DT-DTDftGrn,"{'idx2v': {0: '_padding', 1: '_missing', 2: 'Y..."


In [28]:
fldgrn_folder = 'data/ProcData/FldGrnInfo'

if not os.path.exists(fldgrn_folder): os.makedirs(fldgrn_folder)

fullfldgrn_file = os.path.join(fldgrn_folder, RecName + '.p')
df_FieldGrainInfo.to_pickle(fullfldgrn_file)